In [4]:
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import seaborn as sns

%matplotlib inline
plt.style.use('seaborn-v0_8-dark')

In [522]:
class PortfolioRiskAnalysys:
    """Performs all necessary tasks for a porfolio analysys"""

    def __init__(self,securities,years=20, end_year=2024):
        """Initializes values and collects historical data"""
        self.securities=securities
        self.years=years
        self.end_year=end_year

        self.get_data()

    def get_data(self):
        """Gathers historical data"""

        securities=self.securities
        years=self.years
        end_year=self.end_year


        # Creates an empty pandas dataframe
        df=pd.DataFrame()


        for security, id in securities.items():

            # Creates en empty array for each security where data for each year  will be stored
            his_data_array=[] 

            # Loops through every year for each security
            for year in range(end_year-years,end_year):

                # Collects daily closing data for a whole year and stores it in a json file
                response=requests.get(f'https://www.avanza.se/_api/fund-guide/chart/{id[1]}/{year}-01-01/{year}-12-31?raw=true')

                try:
                    # Transforms the data into a Pandas data frame and turns it into a time series
                    data=pd.DataFrame(response.json()['dataSerie'])
                    data.set_index(pd.to_datetime(data.pop("x"),unit="ms",utc=True),inplace=True)

                except requests.exceptions.RequestException as e:
                    print(f"Failed to download data for {security} in {year}: {e}")
                except KeyError as e:
                    print(f"Data structure issue for {security} in {year}: missing key {e}")


                # Stores the yearly data frame in the security array 
                his_data_array.append(data.y)

            # Puts together the array of data frames and stores them in a column of the df
            df[security]=pd.concat(his_data_array)

        df.index=df.index.date # Sets index to date format (instead of exact time)
        df.index.name="date"
        df=df.shift(1) # Shift one since "swedish time", this could also be done by changing time zone.
        df.dropna(inplace=True)
        self.df=df

    def get_spy_data(self):
        end_year=self.end_year
        years=self.years

        spy_data=[]

        for year in range(end_year-years,end_year):
            response=requests.get(f'https://www.avanza.se/_api/fund-guide/chart/159932/{year}-01-01/{year}-12-31?raw=true')
            data=pd.DataFrame(response.json()['dataSerie'])
            data.set_index(pd.to_datetime(data.pop("x"),unit="ms",utc=True),inplace=True)
            spy_data.append(data.y)
        
        spy_df=pd.concat(spy_data)
        spy_df.index=spy_df.index.date # Sets index to date format (instead of exact time)
        spy_df.index.name="date"
        spy_df=spy_df.shift(1) # Shift one since "swedish time", this could also be done by changing time zone.
        spy_df.dropna(inplace=True)
        self.spy=spy_df

    
    def get_omx_data(self):
        end_year=self.end_year
        years=self.years
        
        omx_data=[]

        for year in range(end_year-years,end_year):
            response=requests.get(f'https://www.avanza.se/_api/fund-guide/chart/19002/{year}-01-01/{year}-12-31?raw=true')
            data=pd.DataFrame(response.json()['dataSerie'])
            data.set_index(pd.to_datetime(data.pop("x"),unit="ms",utc=True),inplace=True)
            omx_data.append(data.y)
        
        omx_df=pd.concat(omx_data)
        omx_df.index=omx_df.index.date # Sets index to date format (instead of exact time)
        omx_df.index.name="date"
        omx_df=omx_df.shift(1) # Shift one since "swedish time", this could also be done by changing time zone.
        omx_df.dropna(inplace=True)
        self.omx=omx_df

    def get_portfolio_data(self):
        df=self.df
        for column in df.columns:
            df[column]=df[column]*self.securities[column][0]

        portfolio_df=df.sum(axis=1)
        self.portfolio=portfolio_df


    




    def plot_risk_return(self):
        """Scatter plots annual volatility/risk"""

        # Creates a new, transformed data frame with risk and volatility
        ret=self.df.pct_change().dropna()
        summary=ret.describe().T.loc[:,["mean","std"]]
        
        # Annulaizes daily data
        summary["std"]=summary["std"]*np.sqrt(252)
        summary["mean"]=summary["mean"]*252
        
        # Plots all points
        summary.plot.scatter(x="std",y="mean",s=50,figsize=(12,7),title="Annual Risk to Reward for all assets",)

        # Annotates the points with security name and (volatility, risk) rounded to 2 decimals and shows it
        for security in summary.index:
            plt.annotate(f"{security} ({summary.loc[security, 'std']:.2f} {summary.loc[security, 'mean']:.2f})",xy=(summary.loc[security,"std"]+0.002,summary.loc[security,"mean"]+0.002),size=15)
        plt.xlabel("Annual volatility (std)")
        plt.ylabel("Annual Reward (mean)")
        plt.show()
    
    def plot_correlation(self):
        """Creates a heatmap to display correlation between assets"""
        ret=self.df.pct_change().dropna()
        plt.figure(figsize=(10,7))
        sns.set(font_scale=1.4)
        sns.heatmap(ret.corr(),cmap="Reds",annot=True,annot_kws={"size":15},vmax=0.6)
        plt.show()

    def plot_max_drawdown(self):
        """Creates a bar chart of max drawdowns of all securities"""
        
        # Separates all securities and stores them in a dictionary
        security_dict = {}
        drawdown_dict = {}
        for column in self.df.columns:
            security_data = self.df[[column]].dropna()  # Treat it as a DataFrame, not Series
            security_dict[column] = security_data
            
            # Calculations
            security_dict[column]["log_ret"] = np.log(security_dict[column][column] / security_dict[column][column].shift(1))
            security_dict[column]["cumm_ret"] = np.exp(security_dict[column]["log_ret"].cumsum())
            security_dict[column]["cumm_max"] = security_dict[column]["cumm_ret"].cummax()
            security_dict[column]["drawdown%"] = (security_dict[column]["cumm_max"]-security_dict[column]["cumm_ret"])/security_dict[column]["cumm_max"]
            # Adds security name and drawdown% do the drawdown dictionary
            drawdown_dict[column]=security_dict[column]["drawdown%"].max()*100
        
        # Creates a barchart of maximum drawdown
        plt.figure(figsize=(10, 6))
        plt.bar(drawdown_dict.keys(), drawdown_dict.values(), color='red')
        plt.title(f"Maximum Drawdown of All Securities since {self.end_year-self.years}")
        plt.ylabel("Max Drawdown (%)")
        plt.xlabel("Securities")
        plt.xticks(rotation=45)
        plt.show()

    def plot_portfolio_growth(self):
        df=self.df

        for column in df.columns:
            df[column]=df[column]*self.securities[column][0]

        portfolio_df=self.df.sum(axis=1)
        portfolio_df=portfolio_df.div(portfolio_df.iloc[0])
        portfolio_df.plot(title=f"Portfolio b&h performance since {self.end_year-self.years} - {self.end_year}", figsize=(11,7))
        plt.show()

    def plot_compare_spy(self):
        df=self.df

        self.get_spy_data()
        self.get_portfolio_data()
        portfolio_df=self.portfolio
        spy_df=self.spy
        
        portfolio_df=portfolio_df.div(portfolio_df.iloc[0])
        spy_df=spy_df.div(spy_df.iloc[0])



        plt.figure(figsize=(11,7))
        portfolio_df.plot(label="Portfolio",title=f"Portfolio performance compared to S&P 500 from {self.end_year-self.years} - {self.end_year}")
        spy_df.plot(label="S&P 500")
        
        plt.legend()
        plt.show()
    
    def plot_compare_omx(self):
        df=self.df

        self.get_omx_data()
        omx_df=self.omx


        self.get_portfolio_data()
        portfolio_df=self.portfolio
        portfolio_df=portfolio_df.div(portfolio_df.iloc[0])

        omx_df=omx_df.div(omx_df.iloc[0])



        plt.figure(figsize=(11,7))
        portfolio_df.plot(label="Portfolio",title=f"Portfolio performance compared to OMX 30 from {self.end_year-self.years} - {self.end_year}")
        omx_df.plot(label="OMX30")
        
        plt.legend()
        plt.show()


    def beta(self):
        self.get_omx_data()
        self.get_portfolio_data()
        self.get_spy_data()

        # Create DataFrame with all required data
        df = pd.DataFrame({
            "spy": self.spy,
            "omx": self.omx,
            "portfolio": self.portfolio
        })

        # Drop rows with missing values
        df.dropna(inplace=True)

        # Calculate returns
        df["spy_return"] = df["spy"].pct_change()
        df["portfolio_return"] = df["portfolio"].pct_change()
        df["omx_return"]=df.omx.pct_change()

        # Drop rows with missing values after calculating returns
        df.dropna(inplace=True)

        # Calculate covariance and variance
        spy_covariance = np.cov(df["portfolio_return"], df["spy_return"])[0, 1]
        spy_variance = np.var(df["spy_return"])

        omx_covariance=np.cov(df.portfolio_return,df.omx_return)[0,1]
        omx_variance=np.var(df.omx_return)

        # Calculate beta
        spy_beta = spy_covariance / spy_variance
        omx_beta = omx_covariance / omx_variance


        print(f"SPY Beta: {spy_beta}\nOMX Beta: {omx_beta}")

    def sharp_ratio(self,risk_free_interest_rate= 0.01):
        self.get_portfolio_data()
        portfolio=self.portfolio
        ret=portfolio.pct_change().dropna()

        mean_ret=ret.mean()*252
        std=ret.std()*np.sqrt(252)
        sharp_ratio = (mean_ret-risk_free_interest_rate)/std

        
        return sharp_ratio


    def alpha(self):
        self.get_portfolio_data()
        self.get_spy_data()
        self.get_omx_data()

        portfolio_ret=self.portfolio.pct_change()
        spy_ret=self.spy.pct_change()
        omx_ret=self.omx.pct_change()

        df=pd.DataFrame({
            "portfolio_ret":portfolio_ret,
            "spy_ret":spy_ret,
            "omx_ret":omx_ret


        })
        df.dropna(inplace=True)

        spy_alpha=(df.portfolio_ret-df.spy_ret).mean()*252
        omx_alpha=(df.portfolio_ret-df.omx_ret).mean()*252

        print(f"SPY Alpha: {spy_alpha*100:.2f}%\nOMX30 Alpha: {omx_alpha*100:.2f}%")


        
        
       


            




    


    
        

        



In [523]:
# In array; first argument indicates how many shares you own and the second is the id of the security
stocks = {"apple": [1,3323],
             "atlasco":[25,5235],
             "intel":[2,3658],
             "investor":[16,5247],
             "novo":[4,52300],
             "nvidia":[2,4478],
             #"palantir":[4,1138439], new stock
             #"volvo_car":[15,1296604] new stock

             }

funds= {"bgf_world_tech":[0.87,431],
        "hb_multi_120":[44.44,951762],
        "hsbc_turkey":[4.075,32347],
        "jpm_us_tech":[0.9,951763],
        "nordea_indien":[7.55,43947],
        #"seb_ai":[7.15,1607800]
        
        }
portfolio = stocks.copy()
portfolio.update(funds)

#stockdata=PortfolioRiskAnalysys(stocks)
#funddata=PortfolioRiskAnalysys(funds,years=6)
portfoliodata=PortfolioRiskAnalysys(portfolio,years=6)


In [524]:
#stockdata.plot_correlation()
#stockdata.plot_risk_return()
#funddata.plot_correlation()
#funddata.plot_risk_return()
#portfoliodata.plot_correlation()
#portfoliodata.plot_risk_return()
#dict=portfoliodata.plot_max_drawdown()
#portfoliodata.plot_compare_omx()
#portfoliodata.plot_compare_spy()
#portfoliodata.beta()
#portfoliodata.sharp_ratio()
portfoliodata.alpha()

SPY Alpha: 9.69%
OMX30 Alpha: 8.92%
